## Random IP Geolocation Visualizer

In [1]:
import requests
import math
from ipwhois import IPWhois
from random import randint
from datetime import datetime
from ipyleaflet import Map, Marker
from IPython.display import Image

headers = {
    'accept': 'application/json',
    'content-type': 'application/json'
}

NASA_KEY = ''

Random IP generator - returns a string in the form of `xxx.xxx.xxx.xxx`, where `xxx` is an integer in range [0, 255].

In [2]:
def random_IP():
    return '.'.join([str(randint(0,255)) for i in range(4)])

Generate a random IP address and ask [freegeoip.app](https://freegeoip.app/) for its Geolocation and other information. _Limited to 15k queries per hour._

In [22]:
IP = random_IP()

url = 'https://freegeoip.app/json/' + IP
response = requests.request('GET', url, headers=headers)
lat = response.json()['latitude']
lon = response.json()['longitude']

print(IP, response.json()['country_name'], response.json()['region_name'], response.json()['city'], lat, lon, sep=', ')

115.28.241.174, China, Shandong, Qingdao, 36.066, 120.3694


Display the location (by latitude and longitude) on a map.

In [23]:
%matplotlib inline

center = float(response.json()['latitude']), float(response.json()['longitude'])
m = Map(center=center, zoom=15)
m.add_layer(Marker(location=center, draggable=False));

display(m)

Map(center=[36.066, 120.3694], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Weather Data
_Using coordinates from the randomly generated IP._

Simple 7-day forecast, including weather conditions and max/min temperatures.

In [24]:
mode = 'civillight'
meteo_url = f'http://www.7timer.info/bin/api.pl?lon={lon}&lat={lat}&product={mode}&output=json'
meteo_response = requests.request('GET', meteo_url, headers=headers)

In [25]:
print('\nDate         Max    Min    Condition\n-----------------------------------')
for day in meteo_response.json()['dataseries']:
    temp_range = str(day['temp2m']['max']).rjust(2, ' ') + "°C   " + str(day['temp2m']['min']).rjust(2, ' ') + "°C"
    date_stamp = str(day['date'])[6:] + '.' + str(day['date'])[4:6] + '.' + str(day['date'])[:4]
    print(date_stamp, ' ', temp_range, ' ', day['weather'])


Date         Max    Min    Condition
-----------------------------------
24.05.2020   22°C   18°C   cloudy
25.05.2020   25°C   17°C   cloudy
26.05.2020   25°C   17°C   ishower
27.05.2020   23°C   15°C   pcloudy
28.05.2020   28°C   16°C   pcloudy
29.05.2020   26°C   17°C   cloudy
30.05.2020   21°C   16°C   ishower


### WHOIS Information

Gather information about the randomly generated IP using a WHOIS lookup.

In [26]:
lookup = IPWhois(IP)
results = lookup.lookup_whois()
for u, v in results['nets'][0].items():
    print(u + ':', v)

cidr: 115.28.0.0/15
name: ALISOFT
handle: ZM1015-AP
range: 115.28.0.0 - 115.29.255.255
description: Aliyun Computing Co., LTD
5F, Builing D, the West Lake International Plaza of S&T
No.391 Wen'er Road, Hangzhou, Zhejiang, China, 310099
country: CN
state: None
city: None
address: Beijing, China
postal_code: None
emails: ['ipas@cnnic.cn', 'jiali.jl@alibaba-inc.com', 'anti-spam@list.alibaba-inc.com', 'cloud-cc-sqcloud@list.alibaba-inc.com', 'guowei.pangw@alibaba-inc.com']
created: None
updated: None


### Landsat Imagery

Get a Landsat 8 image for the closest available date and location. Using NASA's Earth [API](https://api.nasa.gov/).

_More about [Landsat 8](https://earthobservatory.nasa.gov/features/LandsatBigData)._

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
landsat_url = f'https://api.nasa.gov/planetary/earth/imagery?lon={lon}&lat={lat}&date={today}&api_key={NASA_KEY}'
landsat_response = requests.request('GET', landsat_url, headers=headers)

In [ ]:
Image(data=landsat_response.content, width=400)